In [16]:
import sys
sys.path.append('/home/rasa/PycharmProjects/reversi-game/')

In [17]:
import torch
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv, sync_envs_normalization

import stable_baselines3.common.callbacks as callbacks_module
from sb3_contrib.common.maskable.evaluation import evaluate_policy as masked_evaluate_policy

# Modify the namespace of EvalCallback directly
callbacks_module.evaluate_policy = masked_evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback

# from sb3_contrib.common.maskable.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
# from stable_baselines3.common.callbacks import EvalCallback

from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.ppo_mask import MaskablePPO

from shutil import copyfile # keep track of generations
from collections import OrderedDict

from gymnasium.spaces import Discrete, Box, Dict, MultiDiscrete
from gymnasium.wrappers import FlattenObservation
import gymnasium.spaces as spaces
from game_logic import Othello
import numpy as np
import os, math
from itertools import cycle

In [23]:
# Settings
SEED = 19
NUM_TIMESTEPS = int(30_000_000)
EVAL_FREQ = int(20_000)
EVAL_EPISODES = int(200)
BEST_THRESHOLD = 0.30 # must achieve a mean score above this to replace prev best self

RENDER_MODE = False # set this to false if you plan on running for full 1000 trials.

LOGDIR = "ppo_masked/v2/4x_64/"

In [24]:
class OthelloEnv(gym.Env):   
    def __init__(self):
        self.game = Othello()
        self.agent_turn = 1
        shape = self.game.board.shape
        self.action_space = Discrete(shape[0] * shape[1])  # sample - [x, y]
        # self.observation_space = Dict({
        #                                 'board' : Box(0, 2, shape=shape, dtype=int), 
        #                                 'player': Discrete(2, start=1)
        #                               })        
        self.observation_space = Box(low=0, high=1, shape=(64*3,), dtype=np.float32)
        self.other_agent = None
        self.reset_othello_gen = self.reset_othello()    
        self.episodes = 0       
        

    def reset_othello(self):
        '''resets game to starting position 
           and also changes starting player alternatively'''
        infinite_player_turn = cycle([1, 2])
        while True:
            game = Othello()
            model_turn = next(infinite_player_turn)
            yield game, model_turn
    
    def change_to_latest_agent(self, agent):
        self.other_agent = agent

    def get_obs(self):
        encoded_board = self.game.get_encoded_state().reshape(-1)
        return encoded_board        
        
    def check_game_ended(self):
        reward = 0
        done = False
        winner = self.game.get_winner()
        if winner is not None:
            self.episodes += 1
            if self.episodes % 10 == 0:
                print(f'Ep done - {self.episodes}.')
            
            done = True
            if winner == self.agent_turn:
                reward = 1
            elif winner == 3 - self.agent_turn: #  other agent turn/figure
                reward = -1
        return reward, done
    
    def render(self):  # todo 
        pass

    def close(self):  # todo
        pass

    def other_agent_play_move(self): 
        obs = self.get_obs()         
        action, _ = self.other_agent.predict(obs, 
                                             action_masks=self.action_masks(),
                                             deterministic=False) 
        game_action = Othello.get_decoded_field(action)
        self.game.play_move(game_action)

    def step(self, action):
        game_action = Othello.get_decoded_field(action)
        self.game.play_move(game_action)

        # do self play
        while self.game.get_winner() is None and self.game.player_turn != self.agent_turn: #  if game hasnt ended do moves if opponent doesnt have one 
            self.other_agent_play_move()

        reward, done = self.check_game_ended()
        info = {}
        truncated = False

                
        # Return step information
        return self.get_obs(), reward, done, truncated, info
    
    def reset(self, *args, **kwargs):
        self.game, self.agent_turn = next(self.reset_othello_gen)
        if self.agent_turn == 2:
            self.other_agent_play_move()
        return self.get_obs(), None

    def action_masks(self):        
        valid_moves = self.game.valid_moves()
    
        mask = np.zeros(self.game.board.shape, dtype=bool)
        
        # Set True for each index in the set
        for index in valid_moves:
            mask[index] = True
        mask.flatten()
        return mask

In [25]:
class SelfPlayCallback(EvalCallback):
    # hacked it to only save new version offrom gymnasium.wrappers import FlattenObservation best model if beats prev self by BEST_THRESHOLD score
    # after saving model, resets the best score to be BEST_THRESHOLD
    def __init__(self, train_env, eval_env, *args, **kwargs):
        super().__init__(eval_env, *args, **kwargs)
        self.best_mean_reward = BEST_THRESHOLD
        self.generation = 0
        self.train_env = train_env
        self.eval_env = eval_env
    def _on_step(self) -> bool:
        # result = super()._on_step() #  eval needs to be masked, its less efficient 
        result = super()._on_step()
        
        if result and self.best_mean_reward > BEST_THRESHOLD:
            self.generation += 1
            print("SELFPLAY: mean_reward achieved:", self.best_mean_reward)
            print("SELFPLAY: new best model, bumping up generation to", self.generation)            
            source_file = os.path.join(LOGDIR, "best_model.zip")
            backup_file = os.path.join(LOGDIR, "history_"+str(self.generation).zfill(4)+".zip")
            copyfile(source_file, backup_file)
            self.best_mean_reward = BEST_THRESHOLD
            agent = self.model.load(source_file)
            self.train_env.unwrapped.change_to_latest_agent(agent)            
            self.eval_env.envs[0].unwrapped.change_to_latest_agent(agent)      
        return result    

In [ ]:
env = OthelloEnv()
env = Monitor(env=env)

print(f'CUDA available: {torch.cuda.is_available()}')
device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

policy_kwargs = {
    'net_arch': {
        'pi': [64, 64, 64, 64],  # Four hidden layers with 64 units each for the policy network
        'vf': [64, 64, 64, 64]   # Four hidden layers with 64 units each for the value network
    }
}

model = MaskablePPO(policy=MaskableActorCriticPolicy, 
                    env=env, 
                    device=device,
                    verbose=1,
                    learning_rate = 0.001,
                    policy_kwargs=policy_kwargs)
starting_model_filepath = LOGDIR + 'random_start_model'
# model = MaskablePPO.load(starting_model_filepath, env=env)
model.save(starting_model_filepath)

start_model_copy = model.load(starting_model_filepath)
env.unwrapped.change_to_latest_agent(start_model_copy)


env_eval = OthelloEnv()
env_eval = Monitor(env=env_eval)

env_eval = DummyVecEnv(env_fns=[lambda: env_eval])
env_eval.envs[0].unwrapped.change_to_latest_agent(start_model_copy)



eval_callback = SelfPlayCallback(
    env,
    env_eval,
    best_model_save_path=LOGDIR,
    log_path=LOGDIR,
    eval_freq=EVAL_FREQ,
    n_eval_episodes=EVAL_EPISODES,
    deterministic=False 
    )


model.learn(total_timesteps=NUM_TIMESTEPS, callback=eval_callback)

CUDA available: False
Using cpu device
Wrapping the env in a DummyVecEnv.


/home/rasa/venv/ml-venv/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:283: UserWarning: Path 'ppo_masked/v2/4x_64' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


Ep done - 10.
Ep done - 20.
Ep done - 30.
Ep done - 40.
Ep done - 50.
Ep done - 60.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 30       |
|    ep_rew_mean     | 0.0294   |
| time/              |          |
|    fps             | 618      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
Ep done - 70.
Ep done - 80.
Ep done - 90.
Ep done - 100.
Ep done - 110.
Ep done - 120.
Ep done - 130.
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 30         |
|    ep_rew_mean          | 0          |
| time/                   |            |
|    fps                  | 494        |
|    iterations           | 2          |
|    time_elapsed         | 8          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.03888946 |
|    clip_fraction     

Ep done - 620.
Ep done - 630.
Ep done - 640.
Ep done - 650.
Ep done - 660.
Ep done - 10.
Ep done - 20.
Ep done - 30.
Ep done - 40.
Ep done - 50.
Ep done - 60.
Ep done - 70.
Ep done - 80.
Ep done - 90.
Ep done - 100.
Ep done - 110.
Ep done - 120.
Ep done - 130.
Ep done - 140.
Ep done - 150.
Ep done - 160.
Ep done - 170.
Ep done - 180.
Ep done - 190.
Ep done - 200.
Eval num_timesteps=20000, episode_reward=0.27 +/- 0.95
Episode length: 30.07 +/- 0.62
----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 30.1       |
|    mean_reward          | 0.265      |
| time/                   |            |
|    total_timesteps      | 20000      |
| train/                  |            |
|    approx_kl            | 0.09723027 |
|    clip_fraction        | 0.56       |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.77      |
|    explained_variance   | 0.1        |
|    learning_rate        | 0.001      |
|    loss      

Ep done - 1170.
Ep done - 1180.
Ep done - 1190.
Ep done - 1200.
Ep done - 1210.
Ep done - 1220.
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 30.1       |
|    ep_rew_mean          | 0.4        |
| time/                   |            |
|    fps                  | 380        |
|    iterations           | 18         |
|    time_elapsed         | 97         |
|    total_timesteps      | 36864      |
| train/                  |            |
|    approx_kl            | 0.11399764 |
|    clip_fraction        | 0.543      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.52      |
|    explained_variance   | -0.0361    |
|    learning_rate        | 0.001      |
|    loss                 | -0.108     |
|    n_updates            | 170        |
|    policy_gradient_loss | -0.0879    |
|    value_loss           | 0.0546     |
----------------------------------------
Ep done - 1230.
Ep done - 1240.
Ep done - 1

...

Ep done - 53590.
Ep done - 53600.
Ep done - 53610.
Ep done - 53620.
Ep done - 53630.
Ep done - 53640.
Ep done - 53650.
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 29.9       |
|    ep_rew_mean          | 0.02       |
| time/                   |            |
|    fps                  | 302        |
|    iterations           | 785        |
|    time_elapsed         | 5308       |
|    total_timesteps      | 1607680    |
| train/                  |            |
|    approx_kl            | 0.38654244 |
|    clip_fraction        | 0.323      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.4       |
|    explained_variance   | 0.236      |
|    learning_rate        | 0.001      |
|    loss                 | 0.0258     |
|    n_updates            | 7840       |
|    policy_gradient_loss | -0.0384    |
|    value_loss           | 0.205      |
----------------------------------------
Ep done - 53660.
Ep 

Ep done - 54140.
Ep done - 54150.
Ep done - 54160.
Ep done - 54170.
Ep done - 54180.
Ep done - 54190.
Ep done - 54200.
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 30         |
|    ep_rew_mean          | 0.35       |
| time/                   |            |
|    fps                  | 303        |
|    iterations           | 793        |
|    time_elapsed         | 5359       |
|    total_timesteps      | 1624064    |
| train/                  |            |
|    approx_kl            | 0.29875377 |
|    clip_fraction        | 0.313      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.387     |
|    explained_variance   | 0.164      |
|    learning_rate        | 0.001      |
|    loss                 | -0.00138   |
|    n_updates            | 7920       |
|    policy_gradient_loss | -0.0397    |
|    value_loss           | 0.181      |
----------------------------------------
Ep done - 54210.
Ep 

Ep done - 54740.
Ep done - 54750.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 30       |
|    ep_rew_mean     | 0.29     |
| time/              |          |
|    fps             | 303      |
|    iterations      | 801      |
|    time_elapsed    | 5409     |
|    total_timesteps | 1640448  |
---------------------------------
Ep done - 54760.
Ep done - 54770.
Ep done - 54780.
Ep done - 54790.
Ep done - 54800.
Ep done - 54810.
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 30.1       |
|    ep_rew_mean          | 0.34       |
| time/                   |            |
|    fps                  | 303        |
|    iterations           | 802        |
|    time_elapsed         | 5415       |
|    total_timesteps      | 1642496    |
| train/                  |            |
|    approx_kl            | 0.31688622 |
|    clip_fraction        | 0.331      |
|    clip_range           | 0.2 

Ep done - 55360.
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 30         |
|    ep_rew_mean          | 0.41       |
| time/                   |            |
|    fps                  | 303        |
|    iterations           | 810        |
|    time_elapsed         | 5461       |
|    total_timesteps      | 1658880    |
| train/                  |            |
|    approx_kl            | 0.33053112 |
|    clip_fraction        | 0.291      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.343     |
|    explained_variance   | 0.102      |
|    learning_rate        | 0.001      |
|    loss                 | -0.0304    |
|    n_updates            | 8090       |
|    policy_gradient_loss | -0.0373    |
|    value_loss           | 0.18       |
----------------------------------------
Ep done - 55370.
Ep done - 55380.
Ep done - 55390.
Ep done - 55400.
Ep done - 16410.
Ep done - 16420.
Ep done - 16430.
Ep 

Ep done - 55850.
Ep done - 55860.
Ep done - 55870.
Ep done - 55880.
Ep done - 55890.
Ep done - 55900.
Ep done - 55910.
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 30        |
|    ep_rew_mean          | 0.15      |
| time/                   |           |
|    fps                  | 303       |
|    iterations           | 818       |
|    time_elapsed         | 5518      |
|    total_timesteps      | 1675264   |
| train/                  |           |
|    approx_kl            | 0.3139801 |
|    clip_fraction        | 0.319     |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.407    |
|    explained_variance   | 0.0795    |
|    learning_rate        | 0.001     |
|    loss                 | -0.0139   |
|    n_updates            | 8170      |
|    policy_gradient_loss | -0.0447   |
|    value_loss           | 0.204     |
---------------------------------------
Ep done - 55920.
Ep done - 55930.
Ep done

In [130]:
obs = env.unwrapped.get_obs()
obs

OrderedDict([('board',
              array([[0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 1, 2, 0, 0, 0],
                     [0, 0, 0, 2, 1, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0]])),
             ('player', 1)])

In [131]:
new_obs = spaces.flatten(env.unwrapped.observation_space, obs)
new_obs

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [133]:
for i in range(15):
    print(model.predict(new_obs))

(array(22), None)
(array(29), None)
(array(41), None)
(array(28), None)
(array(5), None)
(array(46), None)
(array(24), None)
(array(13), None)
(array(40), None)
(array(59), None)
(array(13), None)
(array(45), None)
(array(18), None)
(array(62), None)
(array(51), None)


In [115]:
new_obs

array([0, 2, 0, 1, 0, 0, 0, 0, 2, 0, 2, 2, 2, 1, 0, 1, 0, 2, 2, 2, 2, 2,
       1, 2, 1, 0, 2, 2, 1, 1, 0, 0, 0, 0, 1, 2, 2, 1, 1, 0, 0, 0, 1, 0,
       2, 2, 1, 1, 0, 2, 1, 0, 2, 2, 2, 1, 2, 0, 0, 0, 2, 0, 2, 0, 0, 1])

In [147]:
env_eval = OthelloEnv()
env_eval = Monitor(env=env_eval)
env_eval = FlattenObservation(env_eval)

# env_eval = DummyVecEnv(env_fns=[lambda: env_eval])

In [161]:
model1 = MaskablePPO.load('history_00000385.zip')
model_random = MaskablePPO.load('ppo_masked_selfplay_2/history_00000218.zip')

In [162]:
# env_eval.envs[0].unwrapped.change_to_latest_agent(model1)
env_eval.unwrapped.change_to_latest_agent(model1)

In [37]:
evaluate_policy = masked_evaluate_policy
episode_rewards, episode_lengths = evaluate_policy(
                model_random,
                env_eval,
                n_eval_episodes=100,                
                deterministic=True,
                return_episode_rewards=True,
                warn=True
            )

In [38]:
np.mean(episode_rewards)

0.33

In [163]:
def my_play(env, model, episodes = 100):     
    wins = 0
    for episode in range(1, episodes+1):
        obs, _ = env.reset()        
        done = False
        score = 0 
        
        while not done:
            # env.render()
            action, _ = model.predict(obs, action_masks=env.unwrapped.action_masks(), deterministic=False)            
            obs, reward, done, _, info = env.step(action)            
            score+=reward
        won = env.game.get_winner() == env.agent_turn
        # print(f'Episode:{episode} Score:{score}, play as {env.agent_turn}, won = {won}')
        if won:
            wins+=1
    return wins


In [166]:
my_play(env_eval, model_random, 100)

51

In [176]:
np.append(env_eval.game.board.flatten(), [3, 4])

array([0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 4])